In [31]:
import numpy as np
import pandas as pd

import obspy
import os
import obspy.signal.cross_correlation as cross_correlation

from geopy.distance import great_circle

### Initiate event parameters

In [32]:
mag = 4.3
eventLat = 46.904747
eventLon = 9.124708
eventDepth = 1.4
eventID = '2020vcnjhp'

#The time of the p-arrival at the closest station. Employ a 2 second buffer before the first P-arrival to account
#for 1) the envelope taking maximum inside 1 second window and 2) travel-time errors in the predicted envelopes
firstParrival = 1.89
if firstParrival < 2:
    firstParrival = 0
else:
    firstParrival = firstParrival-2
    
# Window length after the first P arrival. Needed when loading the predicted envelopes (in operations), and 
# both observed and predicted when testing.
windowLength = firstParrival + 4

### Load the observed and predicted envelopes

In [33]:
def calculate_epicentral_distance(stationLat, stationLon, eqLat, eqLon):
    source_coords = (eqLat, eqLon)
    station_coords = (stationLat, stationLon)

    return great_circle(source_coords, station_coords).km

In [34]:
# Loading the observed envelopes
eventDir = 'observedEnvelopesNpy/earthquakes/' + eventID + '/'
observedEnvelopesList = os.listdir(eventDir)
usedStationNames = []
observedEnvelopes = []
for stationEnvelope in observedEnvelopesList:
    usedStationNames.append(stationEnvelope.split('_')[0])
    observedEnvelopes.append(np.load(eventDir +  stationEnvelope))


In [35]:
# Loading the predicted envelopes for stations that have the observed envelopes available
# conditioned on magnitude, distance and site class.

stationMetaData = pd.read_csv('stationInfo.csv')
predictedEnvelopes = []

for station in usedStationNames:
    net, sta = station.split('.')
    
    staInfo = stationMetaData[(stationMetaData['station_network_code']==net) & (stationMetaData['station_code']==sta)]
    
    ec8class = staInfo.station_EC8.values[0]
    if ec8class == 'A' or ec8class == 'B':
        soilType = 'R'
    else:
        soilType = 'S'
    
    stationLatitude = staInfo.station_latitude_deg.values[0]
    stationLongitude = staInfo.station_longitude_deg.values[0]
    epicentralDistance = calculate_epicentral_distance(stationLatitude, stationLongitude, eventLat, eventLon)
    hypocentralDistance = int((np.sqrt(epicentralDistance**2 + eventDepth**2)))
    
    predictedEnvelopeFolder = '/'.join(['envelopeTemplatesNpy/', soilType, str(mag), str(hypocentralDistance)]) + '/' 
    predictedEnvelopes.append(np.load(predictedEnvelopeFolder + 'V_H.npy')/100) #The predicted envelopes are provided in cm/s, we divide by 100 to get m/s

### Calculate the goodness of fit

In [36]:
def calc_correlation(observedEnvelope, predictedEnvelope):
    
    crossCorrStatic = cross_correlation.correlate(observedEnvelope, predictedEnvelope, shift=0)[0]
    
    if crossCorrStatic < 0:
        crossCorrStatic = 0
    
    return crossCorrStatic

In [37]:
def calculate_amplitude_fit(observedEnvelope, predictedEnvelope):
    obsMax = np.max(observedEnvelope)
    predMax = np.max(predictedEnvelope)
    
    aFit = 1-np.sqrt(((obsMax-predMax)**2)/((obsMax+predMax)**2))
    
    return aFit

In [38]:
# Calculate the goodness-of-fit. A minimal amplitude threshold of 0.00005, 
# in either observed or predicted envelope, is employed as a proxy for triggered stations.
stationGoF = []

for j in range(len(usedStationNames)):
    if np.max([observedEnvelopes[j][firstParrival:windowLength], predictedEnvelopes[j][firstParrival:windowLength]]) > 0.00005:
        corrFit = calc_correlation(observedEnvelopes[j][firstParrival:windowLength], predictedEnvelopes[j][firstParrival:windowLength])
        ampFit = calculate_amplitude_fit(observedEnvelopes[j][firstParrival:windowLength], predictedEnvelopes[j][firstParrival:windowLength])
        stationGoF.append(np.sqrt(ampFit*corrFit))


totalGoF = 100*np.average(stationGoF)
print(totalGoF)

80.7093541881
